In [ ]:
# import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv("/Users/shiyili/projects/kaggle/train.csv")

In [ ]:
df[ df['target'] == 0 ]

In [ ]:
# count nan in each (stock_id, date_id) pair
df_nan = df.groupby(['stock_id', 'time_id']).apply(lambda x: x.isnull().sum()).reset_index()
df_nan.columns = ['stock_id', 'time_id', 'nan_count']
df_nan 